# ORGANIZANDO OS DADOS DA MALHA VIÁRIA DE VIÇOSA - SEM O ANEL

### IMPORTANDO O PACOTE psycopg2 QUE CONECTA O PYTHON COM O POSTGRE-SQL

In [8]:
import psycopg2 as pg
#import pandas as pd

### CONECTANDO AO BANCO DE DADOS

In [9]:
con = pg.connect(host='localhost', 
                database='dissertacao',
                user='postgres', 
                password='admin')

cur = con.cursor() #CRIANDO UMA INSTÂNCIA PARA EXECUTAR COMANDOS EM SQL

# OBS: O servidor hospedado na máquina local será conectado no banco de dados nomeado DISSERTAÇÃO, que possui usuário postgres e senha admin.

### VENDO QUANTOS GRAFOS A MALHA POSSUI

In [10]:
tabela_grafos = 'vias_grafos' #TABELA COM OS GRAFOS
sql = f'select max(id) from {tabela_grafos}' #COMANDO EM SQL A SER EXECUTADO
cur.execute(sql) #EXECUTANDO O COMANDO CRIADO
dados_consultados = cur.fetchall() #RETORNANDO OS DADOS
id_max = dados_consultados[0][0] #ID MÁXIMO DA MALHA

### ATRIBUINDO NOME DAS VIAS PARA O BANCO DE DADOS DOS GRAFOS

In [ ]:
#NOME DAS TABELAS E DOS ATRIBUTOS DA CONSULTA:
tabela_grafos = 'vias_grafos' #NOME DE TABELA QUE CONTÉM O SHP DOS GRAFOS DA MALHA
abr_grafos = 'vg' #ABREVIAÇÃO PARA A TABELA QUE CONTÉM O SHP DOS GRAFOS DA MALHA
atributo_dados_grafo = 'nome_rua' #NOME DA COLUNA (EM BRANCO) QUE SERÃO ADICIONADAS AS INFORMAÇÕES DE NOME DAS RUAS NA TABELA DO SHP DOS GRAFOS
tabela_dados = 'vias_nomeruas' #NOME DE TABELA QUE CONTÉM O SHP DOS NOMES DAS RUAS DA MALHA
abr_dados = 'vnr' #ABREVIAÇÃO PARA A TABELA QUE CONTÉM O SHP DOS NOMES DAS RUAS DA MALHA
atributo_dados = 'nome_rua' #NOME DA COLUNA QUE CONTEM AS INFORMAÇÕES DOS NOMES DAS RUAS NA TABELA DO SHP DOS NOMES DAS RUAS DA MALHA

#ADICIONANDO O NOME DAS RUAS PARA CADA ID DA TABELA DOS GRAFOS DA MALHA

for id_i in range(1, id_max + 1): #OS COMANDOS SERÃO EXECUTADOS DO GRAFO DE ID=1 ATÉ O ÚLTIMO GRAFO DA TABELA QUE POSSUI ID = ID_MAX
  
    sql = f'select {abr_dados}.{atributo_dados} from "{tabela_dados}" {abr_dados}, "{tabela_grafos}" {abr_grafos} where st_contains({abr_grafos}.geom, {abr_dados}.geom) and {abr_grafos}.id = {id_i}' #COMANDO EM SQL A SER EXECUTADO. SERÁ SELECIONADO O ATRIBUTOS INDICADO DA TABELA DOS SHP DOS ATRIBUTOS, ONDE A GEOMETRIA DOS ELEMENTOS DA TABELA DE ATRIBUTOS ESTÁ CONTIDA NA GEOMETRIA DA TABELA DO GRAFO, PARA O ID DO GRAFO EM ANÁLISE.
    
    cur.execute(sql) #EXECUTANDO O COMANDO
    
    dados_consultados = cur.fetchall() #RETORNANDO OS DADOS
    
    lista_dados = [] #CRIANDO UMA LISTA VAZIA, NO QUAL OS DADOS CONSULTADOS SERÃO ADICIONADOS
    
    for dado in dados_consultados: #ADICIONANDO OS RESULTADOS NA LISTA CRIADA
        if dado[0] not in lista_dados:
            lista_dados.append(dado[0])
    
    lista_dados_str = '' #CRIANDO UMA VARIAVEL NO QUAL A LISTA DE DADOS SERÃO TRANSFORMADA EM UMA STRING DA SEGUINTE FORMA: ATRIBUTO_1; ATRIBUTO_2; ATRIBUTO_3; ... ; ATRIBUTO_n
    
    for dado in lista_dados: #TRANSFORMANDO A LISTA DE DADOS PARA UMA STRING
        if dado != lista_dados[len(lista_dados) - 1]:
            lista_dados_str = lista_dados_str + dado + '; '
        else:
            lista_dados_str = lista_dados_str + dado

    #ATUALIZANDO A TABELA DO SHP DOS GRAFOS COM AS INFORMAÇÕES DOS ATRIBUTOS OBTIDOS:
    
    sql = f"update {tabela_grafos} set {atributo_dados_grafo}='{lista_dados_str}' where id={id_i};" #COMANDO EM SQL A SER EXECUTADO. SERÁ ATRIBUITO A TABELA DO SHP DOS GRAFOS A STRING ACIMA DE ACORDO COM O ID EM ANÁLISE.
    cur.execute(sql) #EXECUTANDO O COMANDO
    con.commit() #FINALIZANDO A EXECUÇÃO DO COMANDO

    print(f'ID: {id_i} {atributo_dados_grafo} OK!') #COMANDO DESNECESSÁRIO. APENAS INDICA A FINALIZAÇÃO DAS REPETIÇÕES.

### ATRIBUINDO O TIPO DA VIA DE ACORDO COM O PLANO DE MOBILIDADE PARA O BANCO DE DADOS DOS GRAFOS

In [ ]:
#NOME DAS TABELAS E DOS ATRIBUTOS DA CONSULTA:
tabela_grafos = 'vias_grafos' #NOME DE TABELA QUE CONTÉM O SHP DOS GRAFOS DA MALHA
abr_grafos = 'vg' #ABREVIAÇÃO PARA A TABELA QUE CONTÉM O SHP DOS GRAFOS DA MALHA
atributo_dados_grafo = 'tipo_pm' #NOME DA COLUNA (EM BRANCO) QUE SERÃO ADICIONADAS AS INFORMAÇÕES DE NOME DAS RUAS NA TABELA DO SHP DOS GRAFOS
tabela_dados = 'vias_tipopm' #NOME DE TABELA QUE CONTÉM O SHP DOS NOMES DAS RUAS DA MALHA
abr_dados = 'vpm' #ABREVIAÇÃO PARA A TABELA QUE CONTÉM O SHP DOS NOMES DAS RUAS DA MALHA
atributo_dados = 'tipo_pm' #NOME DA COLUNA QUE CONTEM AS INFORMAÇÕES DOS NOMES DAS RUAS NA TABELA DO SHP DOS NOMES DAS RUAS DA MALHA

#ADICIONANDO O TIPO DA VIA DE ACORDO COM O PLANO DE MOBILIDADE DAS RUAS PARA CADA ID DA TABELA DOS GRAFOS DA MALHA

for id_i in range(1, id_max + 1): #OS COMANDOS SERÃO EXECUTADOS DO GRAFO DE ID=1 ATÉ O ÚLTIMO GRAFO DA TABELA QUE POSSUI ID = ID_MAX
      
    sql = f'select {abr_dados}.{atributo_dados} from "{tabela_dados}" {abr_dados}, "{tabela_grafos}" {abr_grafos} where st_contains({abr_grafos}.geom, {abr_dados}.geom) and {abr_grafos}.id = {id_i}' #COMANDO EM SQL A SER EXECUTADO. SERÁ SELECIONADO O ATRIBUTOS INDICADO DA TABELA DOS SHP DOS ATRIBUTOS, ONDE A GEOMETRIA DOS ELEMENTOS DA TABELA DE ATRIBUTOS ESTÁ CONTIDA NA GEOMETRIA DA TABELA DO GRAFO, PARA O ID DO GRAFO EM ANÁLISE.
    
    cur.execute(sql) #EXECUTANDO O COMANDO
    
    dados_consultados = cur.fetchall() #RETORNANDO OS DADOS
    
    lista_dados = [] #CRIANDO UMA LISTA VAZIA, NO QUAL OS DADOS CONSULTADOS SERÃO ADICIONADOS
    
    for dado in dados_consultados: #ADICIONANDO OS RESULTADOS NA LISTA CRIADA
        if dado[0] not in lista_dados:
            lista_dados.append(dado[0])
    
    lista_dados_str = '' #CRIANDO UMA VARIAVEL NO QUAL A LISTA DE DADOS SERÃO TRANSFORMADA EM UMA STRING DA SEGUINTE FORMA: ATRIBUTO_1; ATRIBUTO_2; ATRIBUTO_3; ... ; ATRIBUTO_n
    
    for dado in lista_dados: #TRANSFORMANDO A LISTA DE DADOS PARA UMA STRING
        if dado != lista_dados[len(lista_dados) - 1]:
            lista_dados_str = lista_dados_str + dado + '; '
        else:
            lista_dados_str = lista_dados_str + dado

    #ATUALIZANDO A TABELA DO SHP DOS GRAFOS COM AS INFORMAÇÕES DOS ATRIBUTOS OBTIDOS:
    
    sql = f"update {tabela_grafos} set {atributo_dados_grafo}='{lista_dados_str}' where id={id_i};" #COMANDO EM SQL A SER EXECUTADO. SERÁ ATRIBUITO A TABELA DO SHP DOS GRAFOS A STRING ACIMA DE ACORDO COM O ID EM ANÁLISE.
    cur.execute(sql) #EXECUTANDO O COMANDO
    con.commit() #FINALIZANDO A EXECUÇÃO DO COMANDO

    print(f'ID: {id_i} {atributo_dados_grafo} OK!') #COMANDO DESNECESSÁRIO. APENAS INDICA A FINALIZAÇÃO DAS REPETIÇÕES.

### ATRIBUINDO O TIPO DE PAVIMENTO PARA O BANCO DE DADOS DOS GRAFOS

In [ ]:
#NOME DAS TABELAS E DOS ATRIBUTOS DA CONSULTA:
tabela_grafos = 'vias_grafos' #NOME DE TABELA QUE CONTÉM O SHP DOS GRAFOS DA MALHA
abr_grafos = 'vg' #ABREVIAÇÃO PARA A TABELA QUE CONTÉM O SHP DOS GRAFOS DA MALHA
atributo_dados_grafo = 'pavimento' #NOME DA COLUNA (EM BRANCO) QUE SERÃO ADICIONADAS AS INFORMAÇÕES DE NOME DAS RUAS NA TABELA DO SHP DOS GRAFOS
tabela_dados = 'vias_pavimentos' #NOME DE TABELA QUE CONTÉM O SHP DOS NOMES DAS RUAS DA MALHA
abr_dados = 'vp' #ABREVIAÇÃO PARA A TABELA QUE CONTÉM O SHP DOS NOMES DAS RUAS DA MALHA
atributo_dados = 'pavimento' #NOME DA COLUNA QUE CONTEM AS INFORMAÇÕES DOS NOMES DAS RUAS NA TABELA DO SHP DOS NOMES DAS RUAS DA MALHA

#ADICIONANDO O TIPO DE PAVIMENTO DAS RUAS PARA CADA ID DA TABELA DOS GRAFOS DA MALHA

for id_i in range(1, id_max + 1): #OS COMANDOS SERÃO EXECUTADOS DO GRAFO DE ID=1 ATÉ O ÚLTIMO GRAFO DA TABELA QUE POSSUI ID = ID_MAX
    
    sql = f'select {abr_dados}.{atributo_dados} from "{tabela_dados}" {abr_dados}, "{tabela_grafos}" {abr_grafos} where st_contains({abr_grafos}.geom, {abr_dados}.geom) and {abr_grafos}.id = {id_i}' #COMANDO EM SQL A SER EXECUTADO. SERÁ SELECIONADO O ATRIBUTOS INDICADO DA TABELA DOS SHP DOS ATRIBUTOS, ONDE A GEOMETRIA DOS ELEMENTOS DA TABELA DE ATRIBUTOS ESTÁ CONTIDA NA GEOMETRIA DA TABELA DO GRAFO, PARA O ID DO GRAFO EM ANÁLISE.
    
    cur.execute(sql) #EXECUTANDO O COMANDO
    
    dados_consultados = cur.fetchall() #RETORNANDO OS DADOS
    
    lista_dados = [] #CRIANDO UMA LISTA VAZIA, NO QUAL OS DADOS CONSULTADOS SERÃO ADICIONADOS
    
    for dado in dados_consultados: #ADICIONANDO OS RESULTADOS NA LISTA CRIADA
        if dado[0] not in lista_dados:
            lista_dados.append(dado[0])
    
    lista_dados_str = '' #CRIANDO UMA VARIAVEL NO QUAL A LISTA DE DADOS SERÃO TRANSFORMADA EM UMA STRING DA SEGUINTE FORMA: ATRIBUTO_1; ATRIBUTO_2; ATRIBUTO_3; ... ; ATRIBUTO_n
    
    for dado in lista_dados: #TRANSFORMANDO A LISTA DE DADOS PARA UMA STRING
        if dado != lista_dados[len(lista_dados) - 1]:
            lista_dados_str = lista_dados_str + dado + '; '
        else:
            lista_dados_str = lista_dados_str + dado

    #ATUALIZANDO A TABELA DO SHP DOS GRAFOS COM AS INFORMAÇÕES DOS ATRIBUTOS OBTIDOS:
    
    sql = f"update {tabela_grafos} set {atributo_dados_grafo}='{lista_dados_str}' where id={id_i};" #COMANDO EM SQL A SER EXECUTADO. SERÁ ATRIBUITO A TABELA DO SHP DOS GRAFOS A STRING ACIMA DE ACORDO COM O ID EM ANÁLISE.
    cur.execute(sql) #EXECUTANDO O COMANDO
    con.commit() #FINALIZANDO A EXECUÇÃO DO COMANDO

    print(f'ID: {id_i} {atributo_dados_grafo} OK!') #COMANDO DESNECESSÁRIO. APENAS INDICA A FINALIZAÇÃO DAS REPETIÇÕES.


### ATRIBUINDO O SENTIDO DA VIA PARA O BANCO DE DADOS DOS GRAFOS

In [ ]:
#NOME DAS TABELAS E DOS ATRIBUTOS DA CONSULTA:
tabela_grafos = 'vias_grafos' #NOME DE TABELA QUE CONTÉM O SHP DOS GRAFOS DA MALHA
abr_grafos = 'vg' #ABREVIAÇÃO PARA A TABELA QUE CONTÉM O SHP DOS GRAFOS DA MALHA
atributo_dados_grafo = 'oneway' #NOME DA COLUNA (EM BRANCO) QUE SERÃO ADICIONADAS AS INFORMAÇÕES DE NOME DAS RUAS NA TABELA DO SHP DOS GRAFOS
tabela_dados = 'vias_oneway' #NOME DE TABELA QUE CONTÉM O SHP DOS NOMES DAS RUAS DA MALHA
abr_dados = 'vow' #ABREVIAÇÃO PARA A TABELA QUE CONTÉM O SHP DOS NOMES DAS RUAS DA MALHA
atributo_dados = 'oneway' #NOME DA COLUNA QUE CONTEM AS INFORMAÇÕES DOS NOMES DAS RUAS NA TABELA DO SHP DOS NOMES DAS RUAS DA MALHA

#ADICIONANDO SENTIDO DAS RUAS PARA CADA ID DA TABELA DOS GRAFOS DA MALHA

for id_i in range(1, id_max + 1): #OS COMANDOS SERÃO EXECUTADOS DO GRAFO DE ID=1 ATÉ O ÚLTIMO GRAFO DA TABELA QUE POSSUI ID = ID_MAX

    sql = f'select {abr_dados}.{atributo_dados} from "{tabela_dados}" {abr_dados}, "{tabela_grafos}" {abr_grafos} where st_contains({abr_grafos}.geom, {abr_dados}.geom) and {abr_grafos}.id = {id_i}' #COMANDO EM SQL A SER EXECUTADO. SERÁ SELECIONADO O ATRIBUTOS INDICADO DA TABELA DOS SHP DOS ATRIBUTOS, ONDE A GEOMETRIA DOS ELEMENTOS DA TABELA DE ATRIBUTOS ESTÁ CONTIDA NA GEOMETRIA DA TABELA DO GRAFO, PARA O ID DO GRAFO EM ANÁLISE.
    
    cur.execute(sql) #EXECUTANDO O COMANDO
    
    dados_consultados = cur.fetchall() #RETORNANDO OS DADOS
    
    lista_dados = [] #CRIANDO UMA LISTA VAZIA, NO QUAL OS DADOS CONSULTADOS SERÃO ADICIONADOS
    
    for dado in dados_consultados: #ADICIONANDO OS RESULTADOS NA LISTA CRIADA
        if dado[0] not in lista_dados:
            lista_dados.append(dado[0])
    
    lista_dados_str = '' #CRIANDO UMA VARIAVEL NO QUAL A LISTA DE DADOS SERÃO TRANSFORMADA EM UMA STRING DA SEGUINTE FORMA: ATRIBUTO_1; ATRIBUTO_2; ATRIBUTO_3; ... ; ATRIBUTO_n
    
    for dado in lista_dados: #TRANSFORMANDO A LISTA DE DADOS PARA UMA STRING
        if dado != lista_dados[len(lista_dados) - 1]:
            lista_dados_str = lista_dados_str + dado + '; '
        else:
            lista_dados_str = lista_dados_str + dado

    #ATUALIZANDO A TABELA DO SHP DOS GRAFOS COM AS INFORMAÇÕES DOS ATRIBUTOS OBTIDOS:
    
    sql = f"update {tabela_grafos} set {atributo_dados_grafo}='{lista_dados_str}' where id={id_i};" #COMANDO EM SQL A SER EXECUTADO. SERÁ ATRIBUITO A TABELA DO SHP DOS GRAFOS A STRING ACIMA DE ACORDO COM O ID EM ANÁLISE.
    cur.execute(sql) #EXECUTANDO O COMANDO
    con.commit() #FINALIZANDO A EXECUÇÃO DO COMANDO

    print(f'ID: {id_i} {atributo_dados_grafo} OK!') #COMANDO DESNECESSÁRIO. APENAS INDICA A FINALIZAÇÃO DAS REPETIÇÕES.


### ATRIBUINDO A LARGURA DA VIA PARA O BANCO DE DADOS DOS GRAFOS

In [ ]:
#NOME DAS TABELAS E DOS ATRIBUTOS DA CONSULTA:
tabela_grafos = 'vias_grafos' #NOME DE TABELA QUE CONTÉM O SHP DOS GRAFOS DA MALHA
abr_grafos = 'vg' #ABREVIAÇÃO PARA A TABELA QUE CONTÉM O SHP DOS GRAFOS DA MALHA
atributo_dados_grafo = 'largura_me' #NOME DA COLUNA (EM BRANCO) QUE SERÃO ADICIONADAS AS INFORMAÇÕES DE NOME DAS RUAS NA TABELA DO SHP DOS GRAFOS
tabela_dados = 'vias_largura' #NOME DE TABELA QUE CONTÉM O SHP DOS NOMES DAS RUAS DA MALHA
abr_dados = 'vl' #ABREVIAÇÃO PARA A TABELA QUE CONTÉM O SHP DOS NOMES DAS RUAS DA MALHA
atributo_dados = 'largura' #NOME DA COLUNA QUE CONTEM AS INFORMAÇÕES DOS NOMES DAS RUAS NA TABELA DO SHP DOS NOMES DAS RUAS DA MALHA

#ADICIONANDO A LARGURA DAS RUAS PARA CADA ID DA TABELA DOS GRAFOS DA MALHA

for id_i in range(1, id_max + 1): #OS COMANDOS SERÃO EXECUTADOS DO GRAFO DE ID=1 ATÉ O ÚLTIMO GRAFO DA TABELA QUE POSSUI ID = ID_MAX

    sql = f'select {abr_dados}.{atributo_dados}, st_length({abr_dados}.geom) from "{tabela_dados}" {abr_dados}, "{tabela_grafos}" {abr_grafos} where st_contains({abr_grafos}.geom, {abr_dados}.geom) and {abr_grafos}.id = {id_i}' #COMANDO EM SQL A SER EXECUTADO. SERÁ SELECIONADO O ATRIBUTOS INDICADO DA TABELA DOS SHP DOS ATRIBUTOS, ONDE A GEOMETRIA DOS ELEMENTOS DA TABELA DE ATRIBUTOS ESTÁ CONTIDA NA GEOMETRIA DA TABELA DO GRAFO, PARA O ID DO GRAFO EM ANÁLISE.
    
    cur.execute(sql) #EXECUTANDO O COMANDO
    
    dados_consultados = cur.fetchall() #RETORNANDO OS DADOS

    numerador = 0 #CRIANDO UMA LISTA VAZIA, NO QUAL SERÁ SOMADO TODOS OS ATRIBUTOS PONDERADOS PELO TAMANHO DE SUA LINHA
    denominador = 0 #SOMA DOS PESOS

    for dado in dados_consultados: #ADICIONANDO OS RESULTADOS NAS LISTAS CRIADAS
        numerador = numerador + dado[0]*dado[1]
        denominador = denominador + dado [1]

    largura_me_i = float(f'{numerador/denominador:.2f}') #CALCULANDO A LARGURA MEDIA DO GRAFO i

    #ATUALIZANDO A TABELA DO SHP DOS GRAFOS COM AS INFORMAÇÕES DOS ATRIBUTOS OBTIDOS:
    
    sql = f"update {tabela_grafos} set {atributo_dados_grafo}='{largura_me_i}' where id={id_i};" #COMANDO EM SQL A SER EXECUTADO. SERÁ ATRIBUITO A TABELA DO SHP DOS GRAFOS A STRING ACIMA DE ACORDO COM O ID EM ANÁLISE.
    cur.execute(sql) #EXECUTANDO O COMANDO
    con.commit() #FINALIZANDO A EXECUÇÃO DO COMANDO

    print(f'ID: {id_i} {largura_me_i} OK!') #COMANDO DESNECESSÁRIO. APENAS INDICA A FINALIZAÇÃO DAS REPETIÇÕES.


### ATRIBUINDO A INCLINAÇÃO MÉDIA PARA O BANCO DE DADOS DOS GRAFOS

In [4]:
#NOME DAS TABELAS E DOS ATRIBUTOS DA CONSULTA:
tabela_grafos = 'vias_grafos' #NOME DE TABELA QUE CONTÉM O SHP DOS GRAFOS DA MALHA
abr_grafos = 'vg' #ABREVIAÇÃO PARA A TABELA QUE CONTÉM O SHP DOS GRAFOS DA MALHA
atributo_dados_grafo = 'decliv_me' #NOME DA COLUNA (EM BRANCO) QUE SERÃO ADICIONADAS AS INFORMAÇÕES DE NOME DAS RUAS NA TABELA DO SHP DOS GRAFOS
tabela_dados = 'mde_vicosa' #NOME DE TABELA QUE CONTÉM O SHP DOS NOMES DAS RUAS DA MALHA
abr_dados = 'mde' #ABREVIAÇÃO PARA A TABELA QUE CONTÉM O SHP DOS NOMES DAS RUAS DA MALHA


#ADICIONANDO A DECLIVIDADE DAS RUAS PARA CADA ID DA TABELA DOS GRAFOS DA MALHA

for id_i in range(1, id_max + 1): #OS COMANDOS SERÃO EXECUTADOS DO GRAFO DE ID=1 ATÉ O ÚLTIMO GRAFO DA TABELA QUE POSSUI ID = ID_MAX
    
    #COMPRIMENTO DO GRAFO

    sql = f'SELECT ST_Length(geom) FROM {tabela_grafos} WHERE id = {id_i}' #COMANDO EM SQL PARA CONSULTAR O COMPRIMENTO DO GRAFO

    cur.execute(sql) #EXECUTANDO O COMANDO

    dados_consultados = cur.fetchall() #RETORNANDO OS DADOS

    comp_grafo = dados_consultados[0][0] #COMPRIMENTO DO GRAFO

    #SELECIONANDO AS ALTITUDES AO LONGO DO GRAFO DE ACORDO COM O COMPRIMENTO DO GRAFO:
    
    if comp_grafo <= 300: #CASO O COMPRIMENTO DO GRAFO SEJA <250m, O GRAFO SERÁ DIVIDIDO EM 4 PARTES:
        
        lista_dados = [] #CRIANDO UMA LISTA VAZIA, NO QUAL OS DADOS CONSULTADOS SERÃO ADICIONADOS
        
        for per in range(0, 101, 25):            

            sql = f"SELECT ST_Value({abr_dados}.rast, pts.geom) as elevacao FROM (SELECT ST_LineInterpolatePoint(ST_LineMerge(geom), {per/100}) as geom FROM {tabela_grafos} WHERE id = {id_i}) as pts, {tabela_dados} {abr_dados} WHERE ST_Intersects({abr_dados}.rast, pts.geom);" #COMANDO EM SQL A SER EXECUTADO. SERÁ SELECIONADO AS COTAS DOS PONTOS AO LONGO DO GRAFO PARA O ID DO GRAFO EM ANÁLISE.

            cur.execute(sql) #EXECUTANDO O COMANDO

            dados_consultados = cur.fetchall() #RETORNANDO OS DADOS            
            
            lista_dados.append(dados_consultados[0][0])    

        #CALCULANDO A DECLIVIDADE MEDIA DO GRAFO
        decliv = 0
        for i in range(0, len(lista_dados)-1):
            decliv = decliv + (lista_dados[i+1] - lista_dados[i])/(comp_grafo/4)

        decliv = round(decliv/4*100, 2)
        
    elif comp_grafo <= 600: #SE O COMPRIMENTO DO GRAFO FOR <600m ENTÃO O GRAFO SERÁ DIVIDIDO EM 10 PARTES:
        
        lista_dados = [] #CRIANDO UMA LISTA VAZIA, NO QUAL OS DADOS CONSULTADOS SERÃO ADICIONADOS
        
        for per in range(0, 101, 10):            

            sql = f"SELECT ST_Value({abr_dados}.rast, pts.geom) as elevacao FROM (SELECT ST_LineInterpolatePoint(ST_LineMerge(geom), {per/100}) as geom FROM {tabela_grafos} WHERE id = {id_i}) as pts, {tabela_dados} {abr_dados} WHERE ST_Intersects({abr_dados}.rast, pts.geom);" #COMANDO EM SQL A SER EXECUTADO. SERÁ SELECIONADO AS COTAS DOS PONTOS AO LONGO DO GRAFO PARA O ID DO GRAFO EM ANÁLISE.

            cur.execute(sql) #EXECUTANDO O COMANDO

            dados_consultados = cur.fetchall() #RETORNANDO OS DADOS            
            
            lista_dados.append(dados_consultados[0][0])    

        #CALCULANDO A DECLIVIDADE MEDIA DO GRAFO
        decliv = 0
        for i in range(0, len(lista_dados)-1):
            decliv = decliv + (lista_dados[i+1] - lista_dados[i])/(comp_grafo/10)

        decliv = round(decliv/10*100, 2)
        
    else:     #SE O COMPRIMENTO DO GRAFO FOR >600m ENTÃO O GRAFO SERÁ DIVIDIDO EM 20 PARTES:
           
        lista_dados = [] #CRIANDO UMA LISTA VAZIA, NO QUAL OS DADOS CONSULTADOS SERÃO ADICIONADOS
        
        for per in range(0, 101, 5):            

            sql = f"SELECT ST_Value({abr_dados}.rast, pts.geom) as elevacao FROM (SELECT ST_LineInterpolatePoint(ST_LineMerge(geom), {per/100}) as geom FROM {tabela_grafos} WHERE id = {id_i}) as pts, {tabela_dados} {abr_dados} WHERE ST_Intersects({abr_dados}.rast, pts.geom);" #COMANDO EM SQL A SER EXECUTADO. SERÁ SELECIONADO AS COTAS DOS PONTOS AO LONGO DO GRAFO PARA O ID DO GRAFO EM ANÁLISE.

            cur.execute(sql) #EXECUTANDO O COMANDO

            dados_consultados = cur.fetchall() #RETORNANDO OS DADOS            
            
            lista_dados.append(dados_consultados[0][0])    


        #CALCULANDO A DECLIVIDADE MEDIA DO GRAFO
        decliv = 0
        for i in range(0, len(lista_dados)-1):
            decliv = decliv + (lista_dados[i+1] - lista_dados[i])/(comp_grafo/20)

        decliv = round(decliv/20*100, 2)
        
    #ATUALIZANDO A TABELA DO SHP DOS GRAFOS COM AS INFORMAÇÕES DOS ATRIBUTOS OBTIDOS:

    sql = f"update {tabela_grafos} set {atributo_dados_grafo}='{decliv}' where id={id_i};" #COMANDO EM SQL A SER EXECUTADO. SERÁ ATRIBUITO A TABELA DO SHP DOS GRAFOS A STRING ACIMA DE ACORDO COM O ID EM ANÁLISE.
    cur.execute(sql) #EXECUTANDO O COMANDO
    con.commit() #FINALIZANDO A EXECUÇÃO DO COMANDO
    
    print(f'ID: {id_i} {decliv} OK!') #COMANDO DESNECESSÁRIO. APENAS INDICA A FINALIZAÇÃO DAS REPETIÇÕES.
    
cur.close() #ENCERRANDO A INSTÂNCIA CRIADA PARA A EXECUÇÃO DO COMANDO
con.close() #ENCERRANDO A CONEXÃO COM O BANCO DE DADOS


ID: 1 -1.38 OK!
ID: 2 -1.38 OK!
ID: 3 -0.29 OK!
ID: 4 14.27 OK!
ID: 5 2.96 OK!
ID: 6 -6.9 OK!
ID: 7 -15.82 OK!
ID: 8 6.47 OK!
ID: 9 4.99 OK!
ID: 10 -2.53 OK!
ID: 11 8.31 OK!
ID: 12 -1.38 OK!
ID: 13 9.31 OK!
ID: 14 -2.17 OK!
ID: 15 13.45 OK!
ID: 16 -3.15 OK!
ID: 17 10.84 OK!
ID: 18 -1.8 OK!
ID: 19 0.42 OK!
ID: 20 -0.7 OK!
ID: 21 4.2 OK!
ID: 22 -2.29 OK!
ID: 23 3.43 OK!
ID: 24 -3.03 OK!
ID: 25 -1.42 OK!
ID: 26 4.49 OK!
ID: 27 -1.23 OK!
ID: 28 -0.22 OK!
ID: 29 1.69 OK!
ID: 30 -1.36 OK!
ID: 31 3.67 OK!
ID: 32 4.62 OK!
ID: 33 -1.19 OK!
ID: 34 1.48 OK!
ID: 35 2.09 OK!
ID: 36 5.87 OK!
ID: 37 0.63 OK!
ID: 38 1.3 OK!
ID: 39 2.81 OK!
ID: 40 0.54 OK!
ID: 41 14.88 OK!
ID: 42 2.53 OK!
ID: 43 0.17 OK!
ID: 44 6.52 OK!
ID: 45 0.2 OK!
ID: 46 -10.46 OK!
ID: 47 0.5 OK!
ID: 48 7.43 OK!
ID: 49 0.39 OK!
ID: 50 -9.86 OK!
ID: 51 8.72 OK!
ID: 52 0.93 OK!
ID: 53 -2.9 OK!
ID: 54 4.19 OK!
ID: 55 6.13 OK!
ID: 56 -0.23 OK!
ID: 57 9.38 OK!
ID: 58 -0.5 OK!
ID: 59 4.66 OK!
ID: 60 7.58 OK!
ID: 61 -1.38 OK!
ID: 62 -2.52